In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
from pathlib import Path
import sys

import cv2
import dill as pickle
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from PIL import Image
import plotly.graph_objects as go
import pyransac3d as pyrsc
from tqdm import tqdm
import scipy.io
from scipy.spatial import KDTree
import torch
import transforms3d as t3d
import visu3d as v3d

from barrelnet.pointnet.pointnet_utils import PointNetEncoder, feature_transform_reguliarzer
from barrelnet.pointnet.barrelnet import BarrelNet
from barrelnet.pointnet.data import pts2inference_format
from barrelnet.dust3r_utils import save_dust3r_outs, read_dust3r, resize_to_dust3r
from barrelnet.langsam_utils import display_image_with_masks, display_image_with_boxes
from barrelnet.utils import segment_pc_from_mask, get_local_plane_mask, rotate_pts_to_ax, get_surface_line_traces, get_ray_trace, icp_translate
from barrelnet.synthbarrel import get_cyl_endpoints, get_cylinder_surf, monte_carlo_volume_ratio, random_cylinder_surf

In [ ]:
# reconstr_dir = Path("results/barrel4-2sec-first5-reconstr")
# reconstr_dir = Path("results/barrelddt1-reconstr")
# reconstr_dir = Path("results/dive8-barrel-15-04-2sec-middle-reconstr")
reconstr_dir = Path("results/barrel2-5sec-contrast-reconstr")
ptspath = reconstr_dir / "rotatedpts.pickle"
matcylpath = reconstr_dir / "matcyl.mat"

if not ptspath.exists() or not matcylpath.exists():
    raise FileNotFoundError("sad :(")

with open(ptspath, "rb") as f:
    rotpcdict = pickle.load(f)
matcylparams = scipy.io.loadmat(matcylpath)["cylparams"][0]

rotatedpc = v3d.Point3d(p=rotpcdict["p"], rgb=rotpcdict["rgb"])
barrelyes = rotpcdict["isbarrel"]
x1 = matcylparams[0:3]
x2 = matcylparams[3:6]
axis_pred = x2 - x1
if axis_pred[2] < 0:
    axis_pred = -axis_pred
h = np.linalg.norm(axis_pred)
axis_pred /= h
r = matcylparams[6]
c = (x1 + x2) / 2

In [ ]:
print(monte_carlo_volume_ratio(100000, x1, x2, r, 0, 0, 1, 0))

xx, yy, zz = get_cylinder_surf(x1, x2, r)
# need to do this to change scatter point size
randidxs = np.random.choice(np.arange(rotatedpc.shape[0]), 40000, replace=False)
randmask = np.zeros(rotatedpc.shape[0], dtype=bool)
randmask[randidxs] = True
pctrace = rotatedpc[(~barrelyes) & randmask].make_traces()[0]
pctrace["marker"]["size"] = 4.0
pctrace["opacity"] = 0.9
pctracebarrel = rotatedpc[barrelyes & randmask].make_traces()[0]
pctracebarrel["marker"]["size"] = 4.0
pctracebarrel["opacity"] = 0.9
fig = v3d.make_fig()
fig.add_traces([pctrace, pctracebarrel])
fig.add_trace(get_ray_trace(c, axis_pred, length=h, width=8, color="#e81b00", markersize=10))
cylsurf = go.Surface(x=xx, y=yy, z=zz, opacity=0.4, surfacecolor=np.zeros_like(xx), colorscale="oranges")
fig.add_trace(cylsurf)
fig.add_traces(get_surface_line_traces(xx, yy, zz, width=2, step=5, include_horizontal=False))
xmin, xmax = np.min(rotatedpc.p[:, 0]), np.max(rotatedpc.p[:, 0])
ymin, ymax = np.min(rotatedpc.p[:, 1]), np.max(rotatedpc.p[:, 1])
xx, yy = np.meshgrid(np.linspace(xmin, xmax, 10), np.linspace(ymin, ymax, 10))
zz = np.zeros_like(xx)
planesurf = go.Surface(x=xx, y=yy, z=zz, opacity=0.1, colorscale="purples")
# fig.add_trace(planesurf)
# fig.add_traces(get_surface_line_traces(xx, yy, zz))
fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False)
fig.show()